In [1]:
def getDataDict(path='c:/dev/dataset/convert/행정표준용어1.xls') :
    import pandas as pd
    data = pd.read_excel(path, header=1, index_col=0)
    return data.set_index('용어명')['영문약어명'].to_dict() 

In [2]:
def getData(path='c:/dev/dataset/convert/행정표준용어1.xls') :
    import pandas as pd
    return pd.read_excel(path, header=1, index_col=0)

In [3]:
def getNouns(input_) :
    from konlpy.tag import Okt
    tl = Okt().pos(input_)
    rl = []
    tmp =''

    for idx in range(len(tl)) :    
        if tl[idx][1] == 'Noun' :
            if tmp != '' :
                rl.append(tmp)
            rl.append(tl[idx][0])
            tmp = ''
        else :
            tmp += str(tl[idx][0])
    
    if tmp != '' :
        rl.append(tmp)
            
    return rl

In [4]:
def convert_v1(input_) :
    from konlpy.tag import Okt
    data = getData()
    list_ = getNouns(input_)
    result = ''
    for txt in list_ :
        if data[data['용어명'] == txt].empty :
            tmp = changeAbrv_v1(translate_by_papago_api(txt))
            result += tmp + '_'
        else :
            result += data[data['용어명'] == txt].iat[0,3] + '_'
    
    if result[-1] == '_' :
        result = result[:-1]
        
    return result
        

In [5]:
def convert_v2(input_) :
    data = getDataDict()
    list_ = getNouns(input_)
    result = ''
    for txt in list_ :
        if not txt in data :
            tmp,data = changeAbrv_v2(translate_by_papago_api(txt),txt,data)
            result += tmp + '_'
        else :
            result += data[txt] + '_'
    
    if result[-1] == '_' :
        result = result[:-1]
        
    return result
        

In [6]:
def sumStrList(l) :
    res = ''
    for txt in l :
        res += txt
    return res

In [7]:
def convert_v3(input_,data) :
    list_ = getNouns(input_)
    result = ''
    if input_ != '' :
        for idx in range(len(list_),0,-1) :
            tmpTxt = sumStrList(list_[:idx])
            if tmpTxt in data.keys() :
                result += data[tmpTxt] + '_'
                if idx != len(list_) :
                    resTmp,data = convert_v3(sumStrList(list_[idx:]),data)
                    result += resTmp
                break
            if idx == 1 :
                tmp,data = changeAbrv_v2(translate_by_papago_api(tmpTxt),tmpTxt,data)
                result += tmp + '_'
                if idx != len(list_) :
                    resTmp,data = convert_v3(sumStrList(list_[idx:]),data)
                    result += resTmp
                    
                    
        if result[-1] == '_' :
            result = result[:-1]
    
    return result, data
    
    

In [4]:
def convert_v4(input_,inputDict,defDict,newDict) :
    list_ = getNouns(input_)
    result = ''
    if input_ != '' :
        for idx in range(len(list_),0,-1) :
            tmpTxt = sumStrList(list_[:idx])
            if tmpTxt in inputDict.keys() :
                result += inputDict[tmpTxt] + '_'
                if idx != len(list_) :
                    resTmp,inputDict,defDict,newDict = convert_v4(sumStrList(list_[idx:]),inputDict,
                                                                  defDict,newDict)
                    result += resTmp
                break
            elif tmpTxt in defDict.keys() :
                result += defDict[tmpTxt] + '_'
                if idx != len(list_) :
                    resTmp,inputDict,defDict,newDict = convert_v4(sumStrList(list_[idx:]),inputDict,
                                                                  defDict,newDict)
                    result += resTmp
                break
            elif tmpTxt in newDict.keys() :
                result += newDict[tmpTxt] + '_'
                if idx != len(list_) :
                    resTmp,inputDict,defDict,newDict = convert_v4(sumStrList(list_[idx:]),inputDict,
                                                                  defDict,newDict)
                    result += resTmp
                break

            if idx == 1 :
                tmp,newDict = changeAbrv_v2(translate_by_papago_api(tmpTxt),tmpTxt,newDict)
                result += tmp + '_'
                if idx != len(list_) :
                    resTmp,inputDict,defDict,newDict = convert_v4(sumStrList(list_[idx:]),inputDict,
                                                                  defDict,newDict)
                    result += resTmp
                    
                    
        if result[-1] == '_' :
            result = result[:-1]
    
    return result, inputDict, defDict, newDict

In [9]:
def changeAbrv_v1(text) :
    import re

    v = ['a','e','i','o','u']
    tmp = text.lower()[1:]

    for ch in v :
        tmp = tmp.replace(ch,'')

    for ch in tmp:
        dup = ch+ch
        tmp = re.sub(dup, ch, tmp)
    
    tmp = re.sub(' ','',tmp)
    
    return (text[0].lower() + tmp).upper()

In [5]:
def changeAbrv_v2(text,origin,dict_) :
    import re

    v = ['a','e','i','o','u']
    tmp = text.lower()[1:]

    for ch in v :
        tmp = tmp.replace(ch,'')

    for ch in tmp:
        dup = ch+ch
        tmp = re.sub(dup, ch, tmp)
    
    tmp = re.sub(' ','',tmp)
    tmp = (text[0].lower() + tmp).upper()
    
    result = ''
    isFound = False
    
    for idx in range(3,len(tmp)) :
        if not tmp[:idx] in dict_.values() :
            result = tmp[:idx]
            isFound = True
            break
    
    # 마지막까지 단어가 있으면 '_' 추가
    if not isFound :
        if not tmp in dict_.values() :
            result = tmp
        else :
            result = tmp + '_'
            
    # dict updated
    dict_[origin] = result
    
    return result,dict_

In [6]:
def translate_by_selenium(text) :
    from selenium import webdriver
    import urllib.request
    import requests
    import time
    from bs4 import BeautifulSoup
    
    chromedriver_dir = 'C:\dev\driver\chromedriver.exe'
    driver = webdriver.Chrome(chromedriver_dir)

    driver.get("https://translate.google.com/")

    driver.find_element_by_id('sugg-item-en').click()
    driver.find_element_by_id('sugg-item-ko').click()

    driver.find_element_by_id('source').send_keys(text)

    time.sleep(1)
    source = driver.page_source
    soupData = BeautifulSoup(source,'html.parser')
    
    driver.close()

    return soupData.find('span',class_='tlid-translation translation').text

In [7]:
def translate_by_papago_api(text) :
    import os
    import sys
    import urllib.request
    import json
    
    client_id = "bONH_8tRvIGnJbn7o966"
    client_secret = "zuoA87y5rv"
    
    output_ = ''
    encText = urllib.parse.quote(text)
    data = "source=ko&target=en&text=" + encText
    url = "https://openapi.naver.com/v1/papago/n2mt"
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        str2dict = json.loads(response_body.decode('utf-8'))
        output_ = str2dict['message']['result']['translatedText']
    else:
        output_ = "Error Code:" +rescode

    return output_
    

In [130]:
translate_by_papago_api('잠수함')

'Submarine'

In [134]:
translate_by_selenium('잠수함')

'submarine'

In [74]:
test1 = convert_v1('안드로이드백신개발부서')
test1

'ANDRD_VACIN_DEVLOP_DEPT'

In [166]:
test = convert_v2('안드로이드백신개발부서')
test

'AND_VACIN_DEVLOP_DEPT'

In [137]:
from konlpy.tag import Okt
Okt().pos('24시간전적하목록제출규정')

[('24시간', 'Number'), ('전적하목록제출규정', 'Foreign')]

In [138]:
Okt().pos('안드로이드백신개발부서')

[('안드로이드', 'Noun'),
 ('백', 'Modifier'),
 ('신', 'Modifier'),
 ('개발', 'Noun'),
 ('부서', 'Noun')]

In [139]:
Okt().pos('트위터백신개발부서')

[('트위터', 'Noun'), ('백신', 'Noun'), ('개발', 'Noun'), ('부서', 'Noun')]

In [85]:
test = '24시간전적하목록제출규정'

data = getDataDict()
tmp,data = convert_v3(test, data)
tmp

'24HAMR'

In [8]:
def outputDict(dict_,path) :
    import pandas as pd
    
    tmp = pd.DataFrame(list(dict_.items()),columns=['input','output'])
    tmp.to_excel(path,index=False)


In [14]:
def convertFile_v1(input_path,output_path='output') :
    import pandas as pd
    
    input_ = pd.read_excel(input_path)
    if output_path[input_path.rfind('/'):].find('.') == -1 :
        output_path += input_path[input_path.rfind('/'):]
    
    dict_ = getDataDict()
    
    for idx in input_['output'].isnull().index :
        tmp, dict_ = convert_v3(input_.at[idx,'input'],dict_)
        input_.output[idx] = tmp
        
    input_.to_excel(output_path,index=False)
    outputDict(dict_,output_path[:output_path.rfind('/')]+'/dict.xlsx')    
    

In [9]:
def convertFile_v2(inputDict, input_path,output_path='output') :
    import pandas as pd
    
    input_ = pd.read_excel(input_path)
    if output_path[input_path.rfind('/'):].find('.') == -1 :
        output_path += input_path[input_path.rfind('/'):]

    defDict = getDataDict()
    newDict = {}
    
    for idx in input_['output'].isnull().index :
        tmp, inputDict,defDict,newDict = convert_v4(input_.at[idx,'input'],inputDict,defDict,newDict)
        input_.output[idx] = tmp
        
    input_.to_excel(output_path,index=False)
    outputDict(newDict,output_path[:output_path.rfind('/')]+'/newDict.xlsx')    
    

In [112]:
import pandas as pd
import numpy as np
input_path = 'input/test.xlsx'

convertFile_v1(input_path)

C:\dev\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [10]:
import pandas as pd
import numpy as np
input_path = 'input/test.xlsx'

inputDict = {}
convertFile_v2(inputDict, input_path)

NameError: name 'sumStrList' is not defined